In [3]:
# Class name: AI181685
# Student code: HE181685
# Student name: Nguyễn Thành Trung
# Subject: CPV301-lAB1

In [4]:
!pip install opencv-python
import cv2
import numpy as np
import math
from tkinter import *

In [5]:
drawing = False
p1 = (0, 0)
p2 = (0, 0)
rectangles = []
def create_background():
    return np.ones((600, 800, 3), dtype=np.uint8) * 255

In [6]:
def draw_rectangle(event, x, y, flags, param):
    global p1, p2, drawing, rectangles, img
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        p1 = (x, y)
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            img = create_background()
            p2 = (x, y)
            cv2.rectangle(img, p1, p2, (0, 0, 255), 2)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        p2 = (x, y)
        cv2.rectangle(img, p1, p2, (0, 0, 255), 2)  
        rectangles.append((p1, p2))
current_angle = 0
current_tx = 0  
current_ty = 0  
current_sx = 1  
current_sy = 1  

In [7]:
def translate():
    global img, current_tx, current_ty
    if rectangles:
        tx = int(translation_x_entry.get())
        ty = int(translation_y_entry.get())
        current_tx += tx 
        current_ty -= ty
        new_rect = apply_translation(rectangles[-1], current_tx, current_ty)  
        img = create_background()
        cv2.rectangle(img, new_rect[0], new_rect[1], (0, 255, 0), 2)

In [8]:
def apply_translation(rect, tx, ty):
    p1, p2 = rect
    p1_new = (p1[0] + tx, p1[1] + ty)
    p2_new = (p2[0] + tx, p2[1] + ty)
    return p1_new, p2_new

In [9]:
def rotate():
    global img, current_angle
    if rectangles:
        angle = float(rotation_entry.get())
        current_angle += angle  
        rotated_corners = apply_rotation(rectangles[-1], current_angle)  
        img = create_background()
        pts = np.array(rotated_corners, np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(img, [pts], isClosed=True, color=(255, 0, 0), thickness=2)


In [10]:
def apply_rotation(rect, angle):
    p1, p2 = rect
    center = ((p1[0] + p2[0]) // 2, (p1[1] + p2[1]) // 2) 
    angle_rad = math.radians(angle)
    corners = [
        p1,               
        (p1[0], p2[1]),   
        p2,                
        (p2[0], p1[1]) ]   
    def rotate_point(point):
        x, y = point
        x_new = center[0] + int((x - center[0]) * math.cos(angle_rad) - (y - center[1]) * math.sin(angle_rad))
        y_new = center[1] + int((x - center[0]) * math.sin(angle_rad) + (y - center[1]) * math.cos(angle_rad))
        return x_new, y_new
    rotated_corners = [rotate_point(corner) for corner in corners]
    return rotated_corners

In [11]:
def scale():
    global img, current_sx, current_sy
    if rectangles:
        sx = float(scaling_x_entry.get())
        sy = float(scaling_y_entry.get())
        current_sx *= sx  
        current_sy *= sy
        new_rect = apply_scaling(rectangles[-1], current_sx, current_sy) 
        img = create_background()
        cv2.rectangle(img, new_rect[0], new_rect[1], (0, 255, 255), 2)

In [12]:
def apply_scaling(rect, sx, sy):
    p1, p2 = rect
    center = ((p1[0] + p2[0]) // 2, (p1[1] + p2[1]) // 2)
    def scale_point(point):
        x, y = point
        x_new = center[0] + int((x - center[0]) * sx)
        y_new = center[1] + int((y - center[1]) * sy)
        return x_new, y_new
    return scale_point(p1), scale_point(p2)

In [13]:
def create_gui():
    root = Tk()
    root.title("Transformations Input")

    translation_label = Label(root, text="TRANSLATION")
    translation_label.pack()

    translation_x_label = Label(root, text="Translation X:")
    translation_x_label.pack()
    global translation_x_entry
    translation_x_entry = Entry(root)
    translation_x_entry.pack()

    translation_y_label = Label(root, text="Translation Y:")
    translation_y_label.pack()
    global translation_y_entry
    translation_y_entry = Entry(root)
    translation_y_entry.pack()

    translation_button = Button(root, text="Apply", command=translate)
    translation_button.pack()

    rotation_label = Label(root, text="ROTATION")
    rotation_label.pack()

    rotation_angle_label = Label(root, text="Rotation Angle (degrees):")
    rotation_angle_label.pack()
    global rotation_entry
    rotation_entry = Entry(root)
    rotation_entry.pack()

    rotation_button = Button(root, text="Apply", command=rotate)
    rotation_button.pack()

    scaling_label = Label(root, text="SCALING")
    scaling_label.pack()

    scaling_x_label = Label(root, text="Scaling Factor X:")
    scaling_x_label.pack()
    global scaling_x_entry
    scaling_x_entry = Entry(root)
    scaling_x_entry.pack()

    scaling_y_label = Label(root, text="Scaling Factor Y:")
    scaling_y_label.pack()
    global scaling_y_entry
    scaling_y_entry = Entry(root)
    scaling_y_entry.pack()

    scaling_button = Button(root, text="Apply", command=scale)
    scaling_button.pack()
    root.mainloop()

img = create_background()
cv2.namedWindow("Rectangle Drawing")
cv2.setMouseCallback("Rectangle Drawing", draw_rectangle)

In [14]:
import threading
threading.Thread(target=create_gui).start()
while True:
    cv2.imshow("Rectangle Drawing", img)
    key = cv2.waitKey(1) & 0xFF
    if key == 27: 
        break

In [15]:
cv2.destroyAllWindows()